Created on Fri Jan 13 14:20:37 2020
<br>
Group 7
<br>
@authors: E.G., C.L.
<h1>Group 7 - Images sociales<span class="tocSkip"></span>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Environment" data-toc-modified-id="Environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Environment</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Functions</a></span></li></ul></li><li><span><a href="#Pre-processing" data-toc-modified-id="Pre-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pre-processing</a></span><ul class="toc-item"><li><span><a href="#Create-directories" data-toc-modified-id="Create-directories-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create directories</a></span></li><li><span><a href="#Train-test-split-and-read-images" data-toc-modified-id="Train-test-split-and-read-images-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Train-test split and read images</a></span></li></ul></li><li><span><a href="#Build,-save,-and-train-model" data-toc-modified-id="Build,-save,-and-train-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Build, save, and train model</a></span></li></ul></div>

# Introduction
This notebook trains a model to predict Boeing aircraft types on images representing an aircraft interior. The images used for training come from Seatguru social media.

**Pre-processing**<span class="tocSkip"></span><br>
By reading and filtering the CSV file that contains the labels (Int, Ext, Ext-Int, Meal), we get the list of Interior labelled images. Then, the images are copied to directories (one per desired aircraft type), and split into train and test sets. If the data augmentation option is set to `True`, the train set will be enriched with new images (obtained by cropping / (de)zooming / rotating / flipping existing images).

**Model**<span class="tocSkip"></span><br>
We get weights from VGG16 pre-trained model, and add some layers (Conv2D, ReLU, MaxPooling2D, Flatten, and Dense) to predict the target classes (e.g.: 4 classes could be 737, 747, 757, and 777).

**Out**<span class="tocSkip"></span><br>
After training, a folder is created in `Models` repository, containing the model in `h5` format, along with the corresponding labels stored in a `pickle` dictionary.

# Environment
To ensure a proper functioning of this code file, `python 3.6` or later version is required.
## Libraries

In [10]:
import warnings
warnings.filterwarnings('ignore')
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.models import Sequential, Model
from keras import backend as K
import keras
import pandas as pd
import random
import os
from PIL import Image
from shutil import copyfile

%load_ext watermark
%watermark -p keras,tensorflow,PIL

## Parameters

In [11]:
project_path = './../'
seatguru_path = project_path + 'Interpromo2020/All Data/ANALYSE IMAGE/IMG SEATGURU/'
stats_path = project_path + 'ImagesStats/'

model_name = 'Int_Boeing_F3'

# Classes to predict
boeing_planes = ['737', '747', '757', '767', '777', '787']
nb_types = len(boeing_planes)

# Images parameters
size = (224, 224)
greys = False

## Functions

In [12]:
%run g7_functions_for_models_V2.ipynb

# Pre-processing
## Create directories

In [13]:
# Read SEATGURU annotated CSV
df_seat_annot = pd.read_csv(stats_path + 'g7_SEATGURU_annotate.csv', sep=';', engine='python',
                            index_col=None, encoding='utf-8')

# Paths where to create aircraft types folders
crea_path = project_path + 'G7_SEATGURU/Int/Boeing/'
new_paths = [crea_path + 'data_train', crea_path + 'data_test']
print(new_paths[0], new_paths[1])

# Create a directory for each class
create_dirs_seatguru_type(df_seat_annot, project_path, seatguru_path, crea_path, aircraft_types=boeing_planes,
                          view='Int', man='Boeing')

./../G7_SEATGURU/Int/Boeing/data_train ./../G7_SEATGURU/Int/Boeing/data_test
./../G7_SEATGURU/Int/Boeing/737
737: 201 images
./../G7_SEATGURU/Int/Boeing/747
747: 52 images
./../G7_SEATGURU/Int/Boeing/757
757: 44 images
./../G7_SEATGURU/Int/Boeing/767
767: 114 images
./../G7_SEATGURU/Int/Boeing/777
777: 279 images
./../G7_SEATGURU/Int/Boeing/787
787: 160 images


## Train-test split and read images

In [14]:
# Split train and test
split_train_test_seatguru_type(
    new_paths=new_paths, path=crea_path, aircraft_types=boeing_planes)

In [15]:
# Image data generator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = datagen.flow_from_directory(new_paths[0],
                                              target_size=size,
                                              color_mode='rgb',
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=True)

test_generator = datagen.flow_from_directory(new_paths[1],
                                             target_size=size,
                                             color_mode='rgb',
                                             batch_size=32,
                                             class_mode='categorical',
                                             shuffle=True)

Found 592 images belonging to 6 classes.
Found 258 images belonging to 6 classes.


# Build, save, and train model

In [16]:
# Get the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False,
                   input_shape=(size[0], size[1], 1 if greys else 3))
x = base_model.output

# Add layers
x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)  # vector

x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# Last layer used to predict our classes
# Dense has the same number of neurons as the number of classes to predict
predictions = Dense(nb_types, activation='softmax')(x)

# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Don't retrain pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done after setting layers to non-trainable)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',  # chosen metric
                              patience=2,  # number of epochs
                              verbose=1,
                              factor=0.5,
                              min_lr=0.00001)

In [18]:
# Train model
step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=20,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])

Epoch 1/20
18/18 [==============================] - 373s 21s/step - loss: 2.0721 - accuracy: 0.3143 - val_loss: 0.8765 - val_accuracy: 0.2403
Epoch 2/20
18/18 [==============================] - 268s 15s/step - loss: 0.7251 - accuracy: 0.7656 - val_loss: 5.4196 - val_accuracy: 0.2558
Epoch 3/20
18/18 [==============================] - 256s 14s/step - loss: 0.1739 - accuracy: 0.9559 - val_loss: 4.8458 - val_accuracy: 0.2907
Epoch 4/20
18/18 [==============================] - 264s 15s/step - loss: 0.0399 - accuracy: 0.9964 - val_loss: 3.9106 - val_accuracy: 0.3333
Epoch 5/20
18/18 [==============================] - 254s 14s/step - loss: 0.0160 - accuracy: 0.9964 - val_loss: 2.5392 - val_accuracy: 0.3721
Epoch 6/20
18/18 [==============================] - 255s 14s/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 1.3194 - val_accuracy: 0.3643
Epoch 7/20
18/18 [==============================] - 250s 14s/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 2.4856 - val_accuracy: 0.3605

Epoch

In [19]:
# Save model and labels
os.makedirs(project_path + 'Models/' + model_name + '/', exist_ok=True)
save_model_classes(project_path + 'Models/',
                   model_name, train_generator, model)